In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [2]:
df=pd.read_csv('Real_Combine.csv')

In [3]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [4]:
X = df.iloc[:,:-1] #independent
y = df.iloc[:,-1] #dependent

In [5]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 25)): # hidden layers can range between 2 to 24
        model.add(layers.Dense(units = hp.Int('Units_' +str(i), min_value = 32, max_value = 256, step = 32), # min no of neuron in a HL is 32 and max is 256
                                        activation = 'relu')) # AF in HL is relu
        model.add(layers.Dense(1, activation = 'linear')) # this is OP layer with AF as linear
        model.compile(
            optimizer = keras.optimizers.Adam(hp.Choice('Learning_Rate', [1e-2, 1e-3, 1e-4])), #Learning rate is 0.001, 0.0001 and 0.00001
            loss = 'mean_absolute_error',
            metrics = ['mean_absolute_error'])
        return model

In [7]:
tuner = RandomSearch(
    build_model,
    objective = "val_mean_absolute_error", #this is for regression. For classification "val_accuracy"
    max_trials = 5,
    executions_per_trial =3, #totally runs 5 X 3 times = 15 times
    directory = 'my_dir',
    project_name = 'hyperparameter_tuning_using_Keras_Tuner')

In [8]:
tuner.search_space_summary()

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [10]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

24/24 [==============================] - ETA: 0s - loss: 189.4800 - mean_absolute_error: 189.480 - 1s 24ms/step - loss: nan - mean_absolute_error: nan - val_loss: 65.1570 - val_mean_absolute_error: 65.1570
Epoch 2/5
24/24 [==============================] - ETA: 0s - loss: 86.3378 - mean_absolute_error: 86.337 - 0s 9ms/step - loss: nan - mean_absolute_error: nan - val_loss: 64.7129 - val_mean_absolute_error: 64.7129
Epoch 3/5
24/24 [==============================] - ETA: 0s - loss: 68.8821 - me

D:\Anaconda3\lib\site-packages\kerastuner\engine\metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

24/24 [==============================] - ETA: 0s - loss: 121.5652 - mean_absolute_error: 121.565 - 1s 38ms/step - loss: nan - mean_absolute_error: nan - val_loss: 62.9248 - val_mean_absolute_error: 62.9248
Epoch 2/5
24/24 [==============================] - ETA: 0s - loss: 65.0203 - mean_absolute_error: 65.020 - 0s 10ms/step - loss: nan - mean_absolute_error: nan - val_loss: 61.9701 - val_mean_absolute_error: 61.9701
Epoch 3/5
24/24 [==============================] - ETA: 0s - loss: 59.1630 - m

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

24/24 [==============================] - ETA: 0s - loss: 177.8789 - mean_absolute_error: 177.878 - 1s 42ms/step - loss: nan - mean_absolute_error: nan - val_loss: 65.6644 - val_mean_absolute_error: 65.6644
Epoch 2/5
24/24 [==============================] - ETA: 0s - loss: 54.1313 - mean_absolute_error: 54.131 - 0s 9ms/step - loss: nan - mean_absolute_error: nan - val_loss: 64.0763 - val_mean_absolute_error: 64.0763
Epoch 3/5
24/24 [==============================] - ETA: 0s - loss: 59.0140 - me

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

24/24 [==============================] - ETA: 0s - loss: 209.3652 - mean_absolute_error: 209.365 - 1s 40ms/step - loss: nan - mean_absolute_error: nan - val_loss: 189.6697 - val_mean_absolute_error: 189.6697
Epoch 2/5
24/24 [==============================] - ETA: 0s - loss: 204.7117 - mean_absolute_error: 204.711 - 0s 9ms/step - loss: nan - mean_absolute_error: nan - val_loss: 145.9608 - val_mean_absolute_error: 145.9608
Epoch 3/5
24/24 [==============================] - ETA: 0s - loss: 135.45

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

24/24 [==============================] - ETA: 0s - loss: 327.3621 - mean_absolute_error: 327.362 - 0s 13ms/step - loss: nan - mean_absolute_error: nan - val_loss: 70.6925 - val_mean_absolute_error: 70.6925
Epoch 2/5
24/24 [==============================] - ETA: 0s - loss: 74.3850 - mean_absolute_error: 74.385 - 1s 36ms/step - loss: nan - mean_absolute_error: nan - val_loss: 67.9827 - val_mean_absolute_error: 67.9827
Epoch 3/5
24/24 [==============================] - ETA: 0s - loss: 72.0519 - m

INFO:tensorflow:Oracle triggered exit


In [11]:
tuner.results_summary() #Score or MAE should be less to have better model